# Oscillating cylinder

This notebook will analyze simulations of accelerating flow past an oscillating cylinder, using a no-slip or volume penalized inner boundary.

A python script is provided for parallelized simulation.

# Imports

In [1]:
import numpy as np
import dedalus.public as de
import matplotlib.pyplot as plt
import os
from scipy.special import erf
import time
import logging
root = logging.root
for h in root.handlers: h.setLevel("INFO")
logger = logging.getLogger(__name__)

from dedalus.tools import post
from glob import glob
import h5py

# Functions

In [40]:
def print_group_contents(group):
    if isinstance(group, h5py.Group):
        print(f"Group: {group.name}")
        for name, item in group.items():
            if isinstance(item, h5py.Group):
                print(f"Subgroup: {name}")
                print_group_contents(item)  # Recursively print contents of subgroups
            elif isinstance(item, h5py.Dataset):
                print(f"Dataset: {name}")
    else:
        print("not a group")

def load_data(filename, *dnames, group='/',show=False,flatten=True,sel=None,checkint=True):
    """Load list of arrays given names of group in an hdf5 file.
    
    Parameters
    ----------
    dnames: list
        strings of dataset names
    filename: string
        file name
    group: string, optional
        subgroup of hdf5 file to write to
    overwrite: boolean, optional
    show: boolean, optional
    flatten: boolean, optional
        return number if single value
    sel: slice object, optional
        return slice of data array
    Returns
    -------
    List of numpy arrays

    """

    with h5py.File(filename,'r') as f:
        arrs = []
        g = f[group]
        print_group_contents(g)
        print(dnames,"loop through dnames")
        for dname in dnames:
            if not sel: sel = Ellipsis
            if show: print(dname,sel)  
            print_group_contents(g[dname])  
            arr = g[dname][sel]
            print(arr.shape,arr.dtype,flatten,np.prod(arr.shape),max(arr.shape))
            if flatten:
                if arr.size == 1: 
                    arr = arr.item()
                elif np.prod(arr.shape) == max(arr.shape): 
                    print('flattening')
                    arr = arr.flatten()  
                    print(arr.shape)              
                elif arr.shape[0] == 1: 
                    arr = arr[0,Ellipsis]
            if checkint and isinstance(arr,float) and arr.is_integer(): arr = int(arr)
            arrs.append(arr)
    return arrs

def get_keys(filename, group='/'):
    """ Helper to get keys of an hdf5 file/group."""
    with h5py.File(filename, 'r') as f:
        g = f[group]
        keys = sorted(list(g.keys()))
    return keys

# Plotting 2D Polar data
def extend_angle(*arrays):
    """Complete the periodic mesh to remove missing slice in polar pcolormesh."""
    return [np.concatenate([arr,arr[[0],:]],axis=0) for arr in arrays]

def polar_plot(theta2,rr,array,
               fig=None,ax=None,savename=False,dpi=200,colorbar=True,
               return_plot=False,wrap=True,**kwargs):
    """Wrapper to create a polar plot of a quantity."""
    if fig==None: fig, ax = plt.subplots(figsize=(4,6),subplot_kw=dict(projection='polar'))
    if wrap: theta2, rr, array = extend_angle(theta2,rr,array)
    plot = ax.pcolormesh(theta2,rr,array,**kwargs)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    if colorbar: plt.colorbar(plot,ax=ax,orientation='horizontal')
    if savename: plt.savefig(savename,dpi=dpi,bbox_inches='tight')
    if return_plot: return fig, ax, plot
    return fig, ax

# Analysis and plotting

In [45]:
# Merge simulation data
sim = 'cylinder-penalized'
force_dir, params_dir, data_dir = glob(os.path.join('runs',sim,'*'))

for folder in [data_dir, force_dir, params_dir]:
    post.merge_analysis(folder,cleanup=True)

2023-12-16 11:26:35,894 post 0/1 INFO :: Merging files from runs/cylinder-penalized/data-cylinder-penalized
2023-12-16 11:26:35,907 post 0/1 INFO :: Merging files from runs/cylinder-penalized/force-cylinder-penalized
2023-12-16 11:26:35,909 post 0/1 INFO :: Merging files from runs/cylinder-penalized/parameters-cylinder-penalized


In [44]:
# Load simulation data
data_file = glob(os.path.join(data_dir,'*'))[0]
force_file = glob(os.path.join(force_dir,'*'))[0]
params_file = glob(os.path.join(params_dir,'*'))[0]

t, theta, r = load_data(data_file,'sim_time','theta/1.0','r/1.0',group='scales',show=True)
us,vs,ps,qs = load_data(data_file,'u','v','p','q',group='tasks',show=True)
gamma, = load_data(params_file,'gamma',group='tasks')

theta2 = theta[:,None] + 0*r[None,:]
rr = 0*theta[:,None] + r[None,:]

Group: /scales
Dataset: (T,T,T)r
Dataset: constant
Dataset: iteration
Dataset: ktheta
Dataset: r
Dataset: sim_time
Dataset: theta
Dataset: timestep
Dataset: wall_time
Dataset: world_time
Dataset: write_number
('sim_time', 'theta', 'r') loop through dnames
sim_time Ellipsis
not a group
(1001,) float64 True 1001 1001
flattening
(1001,)
theta Ellipsis
not a group
(0,) float64 True 0 0
flattening
(0,)
r Ellipsis
not a group
(0,) float64 True 0 0
flattening
(0,)
Group: /tasks
Dataset: Fpx
Dataset: Fpy
Dataset: Fvx
Dataset: Fvy
Dataset: Tv
Dataset: alpha
Dataset: omega
Dataset: phi
('u', 'v', 'p', 'q') loop through dnames
u Ellipsis


KeyError: "Unable to synchronously open object (object 'u' doesn't exist)"

In [ ]:
# Mask function
polar_plot(theta2,rr,gamma,cmap='Greys',vmin=0,vmax=1)

In [ ]:
# True vorticity
polar_plot(theta2,rr,qs[-1]/rr,cmap='PuOr',vmax=5,vmin=-5)

In [ ]:
# True pressure
polar_plot(theta2,rr,ps[-1] - 0.5*(us[-1]**2 + vs[-1]**2),
           cmap='viridis',vmin=-2,vmax=1)